In [1]:
print('waleed')

waleed


In [ ]:
symbol_mem_mappng = {
    'local': 'LCL',
    'argument': 'ARG',
    'this': 'THIS',
    'that': 'THAT',
    'pointer': 'R3',
    'temp': 'R5',
    
}

In [ ]:
import random

C_TYPES = [
    'C_ARITHMETIC',
    'C_PUSH',
    'C_POP',
    'C_LABEL',
    'C_GOTO',
    'C_IF',
    'C_FUNCTION',
    'C_RETURN',
    'C_CALL'
]


def classify_c_type(line):
    '''classifies c_type'''
    command = get_command(line).strip()
    if command in ['add', 'sub', 'neg', 'or', 'and', 'not', 'eq', 'lt', 'gt']:
        return 'C_ARITHMETIC'
    elif command == 'push':
        return 'C_PUSH'
    elif command == 'pop':
        return 'C_POP'
    elif command == 'label':
        return 'C_LABEL'
    elif command == 'goto':
        return 'C_GOTO'
    elif command == 'if-goto':
        return 'C_IF'
    elif command == 'function':
        return 'C_FUNCTION'
    elif command == 'return':
        return 'C_RETURN'
    elif command == 'call':
        return 'C_CALL'
    else:
        raise ValueError(f"Unknown command type: {command}")
    
def get_command(line):
    return line.split(' ')[0]

def get_arg1(line):
    parts = line.split(' ')
    return parts[1] if len(parts) > 1 else None

def get_arg2(line):
    parts = line.split(' ')
    return parts[2] if len(parts) > 2 else None



def loi(op):
    return {
        'add': 'M=D+M',
        'sub': 'M=M-D',
        'neg': 'M=-M',
        'or': 'M=D|M',
        'and': 'M=D&M',
        'not': 'M=!M',
    }[op]

def write_arithmetic(op):
    # Logic for comparison operations (eq, lt, gt)
    if op in ['eq', 'lt' , 'gt']:
        r = random.randint(1, 9999999)
        return f'''
    @SP 
    AM=M-1 
    D=M 
    A=A-1 
    D=M-D
    @if_true_{r}
    D;{'JEQ' if op == 'eq' else 'JLT' if op == 'lt' else 'JGT'}
    @SP
    A=M-1
    M=0
    @END_{r}
    0;JMP
(if_true_{r})
    @SP
    A=M-1
    M=-1
(END_{r})
    '''
    # Corrected logic for unary operations (neg, not)
    if op in ['neg', 'not']:
        return f'''
    @SP
    A=M-1
    {loi(op)}
'''
    # Corrected logic for binary operations (add, sub, and, or)
    else:
        return f'''
    @SP
    AM=M-1
    D=M
    A=A-1
    {loi(op)}
'''
    
def get_segment(arg1):
    return {
        'local': 'LCL',
        'argument': 'ARG',
        'this': 'THIS',
        'that': 'THAT',
        'constant': None,
        'static': None,
        'pointer': None,
        'temp': None,
    }[arg1]
    
    
def write_push(segment, index):
    if segment == 'constant':
        return f'''
    @{index}
    D=A
    @SP
    A=M
    M=D
    @SP
    M=M+1
    '''
    else:   
        return f'''
    @{segment}
    D=M
    @{index}
    A=D+A
    D=M
    @SP
    A=M
    M=D
    @SP
    M=M+1
    '''

def write_pop(segment, index):
    return f'''
    @{segment}
    D=M
    @{index}
    D=D+A
    @R13
    M=D
    @SP
    AM=M-1
    D=M
    @R13
    A=M
    M=D
    '''

def write_push_pop(c_type, arg1, arg2):
    if c_type == 'C_PUSH':
        return write_push(arg1, arg2)
    elif c_type == 'C_POP':
        return write_pop(arg1, arg2)
    else:
        return ''



def clean_line(line):
    '''clean from // and empty lines'''
    return line.split('//')[0].strip()

def is_code_line(line):
    '''check if the line is a code line'''
    return line.strip() and not line.strip().startswith('//')


def add_end():
    return '''
    (END)
        @END
        0;JMP
    '''

def parse(file, output_file):
    with open(file, 'r') as fp, open(output_file, 'w') as out_fp:
        lines = fp.readlines()
        for line in lines:
            if not is_code_line(line):
                continue
            line = clean_line(line)
            op, arg1, arg2 = None, None, None
            c_type = classify_c_type(line)
            if c_type == 'C_RETURN':
                continue
            if c_type == 'C_ARITHMETIC':
                op = get_command(line)
                assembly_code = write_arithmetic(op)
            else:
                arg1 = get_arg1(line)
                if c_type in ['C_PUSH', 'C_POP']:
                    arg2 = get_arg2(line)
                    assembly_code = write_push_pop(c_type, arg1, arg2)
                else:
                    assembly_code = ''
            out_fp.write(assembly_code)
        out_fp.write(add_end())
            

parse('BasicTest.vm', 'BasicTest.asm')

In [ ]:
import random
import os

def get_command_parts(line):
    """Extracts the command and its arguments from a line of VM code."""
    parts = line.split()
    command = parts[0]
    arg1 = parts[1] if len(parts) > 1 else None
    arg2 = int(parts[2]) if len(parts) > 2 else None
    return command, arg1, arg2

def classify_command_type(command):
    """Classifies the command into its C-Type."""
    ARITHMETIC_COMMANDS = ['add', 'sub', 'neg', 'eq', 'gt', 'lt', 'and', 'or', 'not']
    if command in ARITHMETIC_COMMANDS:
        return 'C_ARITHMETIC'
    
    COMMAND_MAP = {
        'push': 'C_PUSH', 'pop': 'C_POP', 'label': 'C_LABEL',
        'goto': 'C_GOTO', 'if-goto': 'C_IF', 'function': 'C_FUNCTION',
        'return': 'C_RETURN', 'call': 'C_CALL'
    }
    
    c_type = COMMAND_MAP.get(command)
    if not c_type:
        raise ValueError(f"Unknown command type: {command}")
    return c_type

def write_arithmetic(command):
    """Generates assembly code for an arithmetic command."""
    if command in ['add', 'sub', 'and', 'or']:
        return _write_binary_op(command)
    elif command in ['neg', 'not']:
        return _write_unary_op(command)
    elif command in ['eq', 'gt', 'lt']:
        return _write_comparison_op(command)

def _write_binary_op(command):
    """Handles add, sub, and, or."""
    op_map = {'add': 'M=D+M', 'sub': 'M=M-D', 'and': 'M=D&M', 'or': 'M=D|M'}
    return f"""
    @SP
    AM=M-1
    D=M
    A=A-1
    {op_map[command]}
"""

def _write_unary_op(command):
    """Handles neg, not."""
    op_map = {'neg': 'M=-M', 'not': 'M=!M'}
    return f"""
    @SP
    A=M-1
    {op_map[command]}
"""

def _write_comparison_op(command):
    """Handles eq, gt, lt."""
    jump_map = {'eq': 'JEQ', 'gt': 'JGT', 'lt': 'JLT'}
    label_suffix = random.randint(1, 999999)
    return f"""
    @SP
    AM=M-1
    D=M
    A=A-1
    D=M-D
    @TRUE_{label_suffix}
    D;{jump_map[command]}
    @SP
    A=M-1
    M=0
    @END_{label_suffix}
    0;JMP
(TRUE_{label_suffix})
    @SP
    A=M-1
    M=-1
(END_{label_suffix})
"""

def _get_address_calc(segment, index, static_filename):
    """Helper to generate assembly for calculating an effective memory address and
       placing it into the A register."""
    if segment in ['local', 'argument', 'this', 'that']:
        segment_map = {'local': 'LCL', 'argument': 'ARG', 'this': 'THIS', 'that': 'THAT'}
        return f"""
    @{segment_map[segment]}
    D=M
    @{index}
    A=D+A""" # A now holds the final address
    elif segment == 'temp':
        return f"""
    @{5 + index}
    A=A""" # A now holds the final address (e.g., @5, A=A)
    elif segment == 'pointer':
        return f"""
    @{3 + index}
    A=A""" # A now holds the final address (e.g., @3, A=A)
    elif segment == 'static':
        return f"""
    @{static_filename}.{index}
    A=A""" # A now holds the final address
    raise ValueError(f"Unknown segment: {segment}")

def write_push(segment, index, static_filename):
    """Generates assembly code for a push operation."""
    if segment == 'constant':
        return f"""
            @{index}
            D=A
            @SP
            A=M
            M=D
            @SP
            M=M+1
        """
    
    # Calculate the address and put it into A
    address_calc_asm = _get_address_calc(segment, index, static_filename)
    
    # Common code: load value from A into D, then push D to stack
    return address_calc_asm + """
    D=M // Load value from RAM[A] into D
    @SP
    A=M
    M=D
    @SP
    M=M+1
"""

def write_pop(segment, index, static_filename):
    """Generates assembly code for a pop operation."""
    # Calculate the address and put it into A
    address_calc_asm = _get_address_calc(segment, index, static_filename)

    # Common code: store the target address (from A) into R13,
    # then pop value from stack into D, then store D into RAM[R13]
    return address_calc_asm + f"""
    D=A // D now holds the target address (RAM[A])
    @R13
    M=D // R13 = target_address
    @SP
    AM=M-1 // SP--, A = new SP
    D=M // D = value popped from stack
    @R13
    A=M // A = target_address
    M=D // RAM[target_address] = popped_value
"""

def write_label(line):
    return '(' + line + ')'

def write_jump(line):
    command, label = line.split(' ')
    if 'IF' in command:
        return f'''
            @SP
            AM=M-1
            D=M
            @{label}
            D;JNE
            '''
    return f'''
        @{label}
        0;JMP
        '''



    
    
    
def clean_line(line):
    """Removes comments and leading/trailing whitespace."""
    return line.split('//')[0].strip()

def add_end_loop():
    """Generates an infinite loop at the end of the assembly file."""
    return """
// Infinite loop at the end
(END)
    @END
    0;JMP
"""

def parse_vm_file(input_file, output_file):
    """Translates a .vm file into a .asm file."""
    static_filename = os.path.basename(output_file).split('.')[0]

    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            cleaned_line = clean_line(line)
            if not cleaned_line:
                continue

            command, arg1, arg2 = get_command_parts(cleaned_line)
            c_type = classify_command_type(command)
            assembly_code = ""

            if c_type == 'C_ARITHMETIC':
                assembly_code = write_arithmetic(command)
            elif c_type == 'C_PUSH':
                assembly_code = write_push(arg1, arg2, static_filename)
            elif c_type == 'C_POP':
                assembly_code = write_pop(arg1, arg2, static_filename)
            elif c_type == 'C_LABEL':
                assembly_code = write_label(line)
            elif c_type in ['C_GOTO', 'C_IF']:
                assembly_code = write_jump(line)
            elif c_type == 'C_FUNCTION':
                ...
            elif c_type == 'C_CALL':
                ...
            elif c_type == 'C_RETURN':
                ...
                
                
                
            outfile.write(f"// {cleaned_line}\n{assembly_code.strip()}\n")
            
        outfile.write(add_end_loop())



# parse_vm_file('StaticTest.vm', 'StaticTest.asm')

In [19]:
print(write_function('function waleed 2'))


            (waleed)
            
                @0
                D=A
                @SP
                A=M
                M=D
                @SP
                M=M+1
                

                @0
                D=A
                @SP
                A=M
                M=D
                @SP
                M=M+1
                
            


# Latest 14/10/2025

In [19]:
import random
import os
import glob
import textwrap

# =================================================================Oo.
#                       PARSING & CLASSIFICATION
# =================================================================Oo.

def get_command_parts(line):
    """Extracts the command and its arguments from a line of VM code."""
    parts = line.split()
    command = parts[0]
    arg1 = parts[1] if len(parts) > 1 else None
    arg2 = int(parts[2]) if len(parts) > 2 else None
    return command, arg1, arg2

def classify_command_type(command):
    """Classifies the command into its C-Type."""
    ARITHMETIC_COMMANDS = ['add', 'sub', 'neg', 'eq', 'gt', 'lt', 'and', 'or', 'not']
    if command in ARITHMETIC_COMMANDS:
        return 'C_ARITHMETIC'

    COMMAND_MAP = {
        'push': 'C_PUSH', 'pop': 'C_POP', 'label': 'C_LABEL',
        'goto': 'C_GOTO', 'if-goto': 'C_IF', 'function': 'C_FUNCTION',
        'return': 'C_RETURN', 'call': 'C_CALL'
    }

    c_type = COMMAND_MAP.get(command)
    if not c_type:
        raise ValueError(f"Unknown command type: {command}")
    return c_type

# =================================================================Oo.
#                         CODE GENERATION
# =================================================================Oo.

def write_arithmetic(command):
    """Generates assembly code for an arithmetic command."""
    if command in ['add', 'sub', 'and', 'or']:
        return _write_binary_op(command)
    elif command in ['neg', 'not']:
        return _write_unary_op(command)
    elif command in ['eq', 'gt', 'lt']:
        return _write_comparison_op(command)

def _write_binary_op(command):
    op_map = {'add': 'M=D+M', 'sub': 'M=M-D', 'and': 'M=D&M', 'or': 'M=D|M'}
    return textwrap.dedent(f"""\
        @SP
        AM=M-1
        D=M
        A=A-1
        {op_map[command]}
    """)

def _write_unary_op(command):
    op_map = {'neg': 'M=-M', 'not': 'M=!M'}
    return textwrap.dedent(f"""\
        @SP
        A=M-1
        {op_map[command]}
    """)

def _write_comparison_op(command):
    jump_map = {'eq': 'JEQ', 'gt': 'JGT', 'lt': 'JLT'}
    label_suffix = random.randint(1, 999999)
    return textwrap.dedent(f"""\
        @SP
        AM=M-1
        D=M
        A=A-1
        D=M-D
        @TRUE_{label_suffix}
        D;{jump_map[command]}
        @SP
        A=M-1
        M=0
        @END_{label_suffix}
        0;JMP
    (TRUE_{label_suffix})
        @SP
        A=M-1
        M=-1
    (END_{label_suffix})
    """)

def _get_address_calc(segment, index, static_filename):
    if segment in ['local', 'argument', 'this', 'that']:
        segment_map = {'local': 'LCL', 'argument': 'ARG', 'this': 'THIS', 'that': 'THAT'}
        return textwrap.dedent(f"""\
            @{segment_map[segment]}
            D=M
            @{index}
            A=D+A
        """)
    elif segment == 'temp':
        return f"@{5 + index}"
    elif segment == 'pointer':
        return f"@{3 + index}"
    elif segment == 'static':
        return f"@{static_filename}.{index}"
    raise ValueError(f"Unknown segment: {segment}")

def write_push(segment, index, static_filename):
    if segment == 'constant':
        return textwrap.dedent(f"""\
            @{index}
            D=A
            @SP
            A=M
            M=D
            @SP
            M=M+1
        """)
    
    address_calc_asm = _get_address_calc(segment, index, static_filename)
    return textwrap.dedent(f"""\
        {address_calc_asm.strip()}
        D=M
        @SP
        A=M
        M=D
        @SP
        M=M+1
    """)

def write_pop(segment, index, static_filename):
    address_calc_asm = _get_address_calc(segment, index, static_filename)
    # Using R13 as a general-purpose temporary register
    return textwrap.dedent(f"""\
        {address_calc_asm.strip()}
        D=A
        @R13
        M=D
        @SP
        AM=M-1
        D=M
        @R13
        A=M
        M=D
    """)

def write_label(label_name):
    return f"({label_name})"

def write_goto(label_name):
    return textwrap.dedent(f"""\
        @{label_name}
        0;JMP
    """)

def write_if_goto(label_name):
    return textwrap.dedent(f"""\
        @SP
        AM=M-1
        D=M
        @{label_name}
        D;JNE
    """)

def write_function(function_name, n_vars):
    # Create a block of assembly for pushing one zero
    init_local_var_code = textwrap.dedent("""\
        @SP
        A=M
        M=0
        @SP
        M=M+1
    """)
    
    # Repeat the push-zero block for each local variable
    initialization_asm = init_local_var_code * n_vars
    
    return f"({function_name})\n{initialization_asm.strip()}"

def write_call(function_name, n_args, call_idx):
    return_address_label = f"{function_name}$ret.{call_idx}"
    return textwrap.dedent(f"""\
        @{return_address_label}
        D=A
        @SP
        A=M
        M=D
        @SP
        M=M+1
        @LCL
        D=M
        @SP
        A=M
        M=D
        @SP
        M=M+1
        @ARG
        D=M
        @SP
        A=M
        M=D
        @SP
        M=M+1
        @THIS
        D=M
        @SP
        A=M
        M=D
        @SP
        M=M+1
        @THAT
        D=M
        @SP
        A=M
        M=D
        @SP
        M=M+1
        @SP
        D=M
        @{n_args}
        D=D-A
        @5
        D=D-A
        @ARG
        M=D
        @SP
        D=M
        @LCL
        M=D
        @{function_name}
        0;JMP
    ({return_address_label})
    """)

def write_return():
    return textwrap.dedent("""\
        @LCL
        D=M
        @R13
        M=D
        @5
        A=D-A
        D=M
        @R14
        M=D
        @SP
        AM=M-1
        D=M
        @ARG
        A=M
        M=D
        @ARG
        D=M+1
        @SP
        M=D
        @R13
        D=M
        @1
        A=D-A
        D=M
        @THAT
        M=D
        @R13
        D=M
        @2
        A=D-A
        D=M
        @THIS
        M=D
        @R13
        D=M
        @3
        A=D-A
        D=M
        @ARG
        M=D
        @R13
        D=M
        @4
        A=D-A
        D=M
        @LCL
        M=D
        @R14
        A=M
        0;JMP
    """)
    
def write_init(call_idx_ref):
    """Writes the bootstrap code to start the program."""
    # A mutable object (list) is used to pass the call counter by reference
    call_sys_init = write_call("Sys.init", 0, call_idx_ref[0])
    call_idx_ref[0] += 1
    
    bootstrap = textwrap.dedent("""\
        @256
        D=A
        @SP
        M=D
    """)
    return f"// Bootstrap Code\n{bootstrap.strip()}\n{call_sys_init}"

# =================================================================Oo.
#                         MAIN TRANSLATOR LOGIC
# =================================================================Oo.

def clean_line(line):
    """Removes comments and leading/trailing whitespace."""
    return line.split('//')[0].strip()

def translate_vm_file(input_vm_file, output_asm_file_handle, call_idx_ref):
    """Translates a SINGLE .vm file and writes to an OPEN .asm file handle."""
    static_filename = os.path.basename(input_vm_file).split('.')[0]

    with open(input_vm_file, 'r') as infile:
        for line in infile:
            cleaned_line = clean_line(line)
            if not cleaned_line:
                continue

            command, arg1, arg2 = get_command_parts(cleaned_line)
            c_type = classify_command_type(command)
            assembly_code = ""

            if c_type == 'C_ARITHMETIC':
                assembly_code = write_arithmetic(command)
            elif c_type == 'C_PUSH':
                assembly_code = write_push(arg1, arg2, static_filename)
            elif c_type == 'C_POP':
                assembly_code = write_pop(arg1, arg2, static_filename)
            elif c_type == 'C_LABEL':
                assembly_code = write_label(arg1)
            elif c_type == 'C_GOTO':
                assembly_code = write_goto(arg1)
            elif c_type == 'C_IF':
                assembly_code = write_if_goto(arg1)
            elif c_type == 'C_FUNCTION':
                assembly_code = write_function(arg1, arg2)
            elif c_type == 'C_CALL':
                assembly_code = write_call(arg1, arg2, call_idx_ref[0])
                call_idx_ref[0] += 1
            elif c_type == 'C_RETURN':
                assembly_code = write_return()
            
            # Write the original VM command as a comment, followed by the assembly code
            output_asm_file_handle.write(f"// {cleaned_line}\n{assembly_code.strip()}\n")

# =================================================================Oo.
#                         SCRIPT EXECUTION
# =================================================================Oo.

def main():
    # --- USER CONFIGURATION: Set your input path here ---
    # It can be a single .vm file or a directory containing .vm files.
    input_path = "hw8/StaticTest/" 

    if os.path.isdir(input_path):
        dir_name = os.path.basename(os.path.normpath(input_path))
        output_path = os.path.join(input_path, f"{dir_name}.asm")
        vm_files_to_process = glob.glob(os.path.join(input_path, "*.vm"))
    elif os.path.isfile(input_path) and input_path.endswith(".vm"):
        output_path = input_path.replace(".vm", ".asm")
        vm_files_to_process = [input_path]
    else:
        print(f"Error: Input path '{input_path}' is not a valid .vm file or directory.")
        return

    if not vm_files_to_process:
        print(f"Error: No .vm files found in '{input_path}'")
        return

    # A list is used to pass the counter by reference across function calls
    call_counter = [0]
    
    with open(output_path, 'w') as outfile:
        # Write bootstrap code only if Sys.vm exists in the directory
        if any("Sys.vm" in f for f in vm_files_to_process):
            outfile.write(write_init(call_counter))
        
        for vm_file in vm_files_to_process:
            print(f"Translating {os.path.basename(vm_file)}...")
            translate_vm_file(vm_file, outfile, call_counter)

    print(f"\nSuccessfully translated to {output_path}")

# This allows the script to be run from the command line
if __name__ == '__main__':
    main()

Translating Class1.vm...
Translating Sys.vm...
Translating Class2.vm...

Successfully translated to hw8/StaticTest/StaticTest.asm


In [5]:
print(write_function('function bar 2'))


(bar)
    @0
    D=A
    @SP
    A=M
    M=D
    @SP
    M=M+1
    @0
    D=A
    @SP
    A=M
    M=D
    @SP
    M=M+1

